In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import koreanize_matplotlib
from datetime import datetime, timedelta
import time
import dateutil
from pathlib import Path

In [ ]:
PATH = r"C:\Users\kmfm1\BIproject\data"

In [ ]:
customer_feedback = pd.read_csv(PATH + r"\customer_feedback.csv")
customers = pd.read_csv(PATH + r"\customers.csv")
delivery_performance = pd.read_csv(PATH + r"\delivery_performance.csv")
inventory = pd.read_csv(PATH + r"\inventory.csv")
inv_new = pd.read_csv(PATH + r"\inventoryNew.csv")
order_items = pd.read_csv(PATH + r"\order_items.csv")
orders = pd.read_csv(PATH + r"\orders.csv")
products = pd.read_csv(PATH + r"\products.csv")
marketing_performance = pd.read_csv(PATH + r"\marketing_performance.csv")

In [ ]:
# 2) 날짜 변환
orders["order_date"] = pd.to_datetime(orders["order_date"])
inventory["date"] = pd.to_datetime(inventory["date"], dayfirst=True)

# 3) 입고 lot
receipts = (
    inventory.groupby(["product_id", "date"], as_index=False)
             .agg(received_qty=("stock_received", "sum"))
)

# 4) 판매 데이터
sales = (
    order_items.merge(orders[["order_id", "order_date"]], on="order_id", how="left")
    [["product_id", "order_date"]]
)

# 5) 입고일 기준 90일 내 주문 존재 여부
def has_sale_within_90(pid, stock_date):
    sale_dates = sales.loc[sales["product_id"] == pid, "order_date"]
    return ((sale_dates >= stock_date) &
            (sale_dates <= stock_date + pd.Timedelta(days=90))).any()

receipts["ordered_within_90"] = receipts.apply(
    lambda row: has_sale_within_90(row["product_id"], row["date"]),
    axis=1
)

# 6) Dead lot
dead_lots = receipts[~receipts["ordered_within_90"]].copy()

# 7) product_name + category 붙이고 누적합
result = (
    dead_lots.merge(products[["product_id", "product_name", "category"]], on="product_id", how="left")
    .groupby(["product_name", "category"], as_index=False)
    .agg(dead_stock_qty=("received_qty", "sum"))
    .sort_values("dead_stock_qty", ascending=False)
)

result

,product_name,category,dead_stock_qty
36,Pet Treats,Pet Care,985
35,Pain Reliever,Pharmacy,736
49,Vitamins,Pharmacy,667
39,Pulses,Grocery & Staples,645
14,Cough Syrup,Pharmacy,600
19,Dog Food,Pet Care,579
46,Toilet Cleaner,Household Care,569
21,Frozen Biryani,Instant & Frozen Food,546
28,Lotion,Personal Care,509
0,Baby Food,Baby Care,474


In [ ]:
# 2) 날짜 변환
orders["order_date"] = pd.to_datetime(orders["order_date"], errors="coerce")
inventory["date"] = pd.to_datetime(inventory["date"], dayfirst=True, errors="coerce")

orders = orders.dropna(subset=["order_date"])
inventory = inventory.dropna(subset=["product_id", "date"])

# 3) 마지막 시점(end_date)
end_date = orders["order_date"].max()
WINDOW = 90
cutoff_stock_date = end_date - pd.Timedelta(days=WINDOW)

# 4) 입고 lot (product_id, date)별 입고수량 합
receipts = (
    inventory.groupby(["product_id", "date"], as_index=False)
             .agg(received_qty=("stock_received", "sum"))
)
receipts = receipts[receipts["received_qty"] > 0].copy()

# ✅ 마지막 시점 기준 90일 이상 지난 lot만 판정 대상
receipts = receipts[receipts["date"] <= cutoff_stock_date].copy()

# 5) 판매일(주문일) 데이터 (product_id별 주문일 리스트)
sales = (
    order_items.merge(orders[["order_id", "order_date"]], on="order_id", how="left")[["product_id", "order_date"]]
    .dropna(subset=["product_id", "order_date"])
)

# product_id -> 정렬된 주문일 numpy array
sales_map = {
    pid: np.sort(g["order_date"].to_numpy(dtype="datetime64[ns]"))
    for pid, g in sales.groupby("product_id")
}

# 6) 입고일~입고일+90일 사이 주문 존재 여부 (빠른 이진탐색)
stock_dates = receipts["date"].to_numpy(dtype="datetime64[ns]")
end_windows = (receipts["date"] + pd.Timedelta(days=WINDOW)).to_numpy(dtype="datetime64[ns]")
pids = receipts["product_id"].to_numpy()

ordered_within_90 = np.zeros(len(receipts), dtype=bool)

for i, (pid, s, e) in enumerate(zip(pids, stock_dates, end_windows)):
    arr = sales_map.get(pid)
    if arr is None or arr.size == 0:
        ordered_within_90[i] = False
        continue
    left = np.searchsorted(arr, s, side="left")
    right = np.searchsorted(arr, e, side="right")
    ordered_within_90[i] = (right > left)

receipts["ordered_within_90"] = ordered_within_90
receipts["Dead_stock"] = ~receipts["ordered_within_90"]

# 7) product_name, category 붙이고 누적합
dead_lots = receipts[receipts["Dead_stock"]].copy()

result = (
    dead_lots.merge(products[["product_id", "product_name", "category"]], on="product_id", how="left")
             .groupby(["product_name", "category"], as_index=False)
             .agg(dead_stock_qty=("received_qty", "sum"))
             .sort_values("dead_stock_qty", ascending=False)
)

print("end_date =", end_date)
result

end_date = 2024-11-04 20:29:15


,product_name,category,dead_stock_qty
32,Pain Reliever,Pharmacy,418
45,Vitamins,Pharmacy,389
20,Frozen Biryani,Instant & Frozen Food,369
33,Pet Treats,Pet Care,364
18,Dog Food,Pet Care,289
42,Toilet Cleaner,Household Care,280
16,Detergent,Household Care,261
1,Baby Wipes,Baby Care,255
0,Baby Food,Baby Care,245
36,Pulses,Grocery & Staples,243


In [ ]:
end_date = orders["order_date"].max()
cutoff = end_date - pd.Timedelta(days=30)

# ------------------
# 2. 최근 90일 판매된 상품(product_id)
# ------------------
recent_sold_pids = set(
    order_items.merge(orders[["order_id", "order_date"]],
                      on="order_id", how="left")
               .query("order_date >= @cutoff")["product_id"]
               .dropna()
               .unique()
)

# ------------------
# 3. 최근 90일 판매되지 않은 상품 추출
# ------------------
result_table = (
    products[~products["product_id"].isin(recent_sold_pids)]
    [["product_id", "product_name", "category"]]
    .copy()
)
# 각 SKU는 1개로 카운트
result_table["not_sold_90d_count"] = 1

# ------------------
# 4. category + product_name 기준 집계
# ------------------
summary = (
    result_table
    .groupby(["category", "product_name"], as_index=False)
    .agg(not_sold_90d_count=("not_sold_90d_count", "sum"))
    .sort_values("not_sold_90d_count", ascending=False)
)

summary.rename(columns={'not_sold_90d_count':'Dead_stock'}, inplace=True)
summary

# ------------------
# 5. price 붙이기
# ------------------
summary = summary.merge(
    products[["product_name", "category", "price"]]
        .drop_duplicates(),
    on=["product_name", "category"],
    how="left"
)


# ------------------
# 6. 손해액 계산
# ------------------
summary["loss_amount"] = summary["price"] * summary["Dead_stock"]

# ------------------
# 7. 정렬
# ------------------
summary = summary.sort_values("loss_amount", ascending=False)
summary


,category,product_name,Dead_stock,price,loss_amount
11,Pet Care,Pet Treats,6,906.54,5439.24
37,Household Care,Toilet Cleaner,5,994.56,4972.80
8,Pet Care,Pet Treats,6,799.46,4796.76
10,Pet Care,Pet Treats,6,787.40,4724.40
32,Household Care,Toilet Cleaner,5,936.98,4684.90
...,...,...,...,...,...
67,Cold Drinks & Juices,Orange Juice,4,16.86,67.44
108,Instant & Frozen Food,Frozen Biryani,3,22.04,66.12
189,Household Care,Dish Soap,2,22.14,44.28
220,Grocery & Staples,Salt,1,37.80,37.80


In [ ]:
# summary.to_csv('Dead_stock.csv', index=False)

In [ ]:
category_summary = (
    summary
    .groupby("category", as_index=False)
    .agg(
        price=("price", "first"),          # price는 그대로 (대표값 1개)
        Dead_stock=("Dead_stock", "sum"),  # SKU 개수 합
        loss_amount=("loss_amount", "sum") # 손해액 합
    )
    .sort_values("loss_amount", ascending=False)
)

category_summary

,category,price,Dead_stock,loss_amount
8,Pet Care,906.54,110,60169.24
9,Pharmacy,877.89,83,47800.29
2,Dairy & Breakfast,951.43,76,42400.56
5,Household Care,994.56,98,38889.35
7,Personal Care,878.07,75,31683.10
1,Cold Drinks & Juices,881.42,53,26490.03
4,Grocery & Staples,778.90,66,24636.39
3,Fruits & Vegetables,728.73,45,23780.99
6,Instant & Frozen Food,827.77,48,19478.81
10,Snacks & Munchies,746.03,32,12378.88


In [ ]:
prodcuts.head()